$\beta = -1$ means that $\beta$ is selected at random in each iteration.

In [1]:
from grasp_iters import read_results

iters = read_results("rl1323_882_441_0")

In [2]:
iters

,iter,beta,RGD_OF,AFVS_OF,time,is_new_best
0,0,0.711551,5629,3547,1.261990,True
1,1,0.625327,4986,3547,2.499386,False
2,2,0.257563,4743,3547,3.859037,False
3,3,0.866414,4495,3547,4.573510,False
4,4,0.029578,5150,3547,5.496411,False
...,...,...,...,...,...,...
4995,4995,0.885710,5801,3547,5957.407021,False
4996,4996,0.992178,5948,3547,5959.041261,False
4997,4997,0.652068,4820,3547,5960.727530,False
4998,4998,0.071028,4958,3547,5961.622590,False


In [3]:
iters[iters["is_new_best"] == True]

,iter,beta,RGD_OF,AFVS_OF,time,is_new_best
0,0,0.711551,5629,3547,1.26199,True
